# Infer Model Parameters for Individuals in Lung Cancer Control Group

## Show control group data

In [1]:
import os

import pandas as pd

import erlotinib as erlo


# Get data
data = erlo.DataLibrary().lung_cancer_control_group()

# Create scatter plot
fig = erlo.plots.PDDataPlot()
fig.add_data(data, id_key='#ID', time_key='TIME in day', pd_key='TUMOUR VOLUME in cm^3')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')

# Show figure
fig.show()

## Find MAP estimates for model parameters

In [2]:
import numpy as np
import pints
from tqdm.notebook import tqdm


# Define model
path = erlo.ModelLibrary().tumour_growth_inhibition_pd_model()
model = erlo.PharmacodynamicModel(path)

# Find maximum a posteriori probability estimates (MAP)
result = pd.DataFrame(columns=['ID', 'Parameter', 'Estimate', 'Score', 'Run'])
ids = data['#ID'].unique()
for individual in tqdm(ids):
    # Get data for individual
    mask = data['#ID'] == individual
    times = data[mask]['TIME in day'].to_numpy()
    observed_volumes = data[mask]['TUMOUR VOLUME in cm^3'].to_numpy()

    # Create inverse problem
    problem = erlo.InverseProblem(model, times, observed_volumes)
    log_likelihood = pints.ConstantAndMultiplicativeGaussianLogLikelihood(problem)
    log_prior_tumour_volume = pints.UniformLogPrior(1E-3, 1E1)
    log_prior_drug_conc = pints.UniformLogPrior(-1E-3, 1E-3)  # Fixed to zero below
    log_prior_kappa = pints.UniformLogPrior(-1E-3, 1E-3)  # Fixed to zero below
    log_prior_lambda_0 = pints.UniformLogPrior(1E-3, 1E1)
    log_prior_lambda_1 = pints.UniformLogPrior(1E-3, 1E1)
    log_prior_sigma_base = pints.HalfCauchyLogPrior(location=0, scale=3)
    log_prior_sigma_rel = pints.HalfCauchyLogPrior(location=0, scale=3)
    log_prior = pints.ComposedLogPrior(
        log_prior_tumour_volume, 
        log_prior_drug_conc,
        log_prior_kappa,
        log_prior_lambda_0,
        log_prior_lambda_1,
        log_prior_sigma_base,
        log_prior_sigma_rel)
    log_posterior = pints.LogPosterior(log_likelihood, log_prior)

    # Set up optmisation controller
    optimiser = erlo.OptimisationController(
        log_posterior=log_posterior,
        optimiser=pints.CMAES)
    
    # Fix potency kappa, and drug concentration to 0
    # (This is the control group, and thus there is no drug)
    optimiser.fix_parameters(
        mask=[False, True, True, False, False, False, False],
        values=[0, 0])

    # Run optimisation
    individual_result = optimiser.run()

    # Append individual result to result
    individual_result['ID'] = individual
    result = result.append(individual_result)

In [3]:
result[result['ID'] == 40]

,ID,Parameter,Estimate,Score,Run
0,40,myokit.tumour_volume,0.137718,18.520070,1
1,40,myokit.lambda_0,0.284953,18.520070,1
2,40,myokit.lambda_1,0.129620,18.520070,1
3,40,noise 1,0.056481,18.520070,1
0,40,myokit.tumour_volume,0.139336,18.526362,2
1,40,myokit.lambda_0,0.275265,18.526362,2
2,40,myokit.lambda_1,0.130952,18.526362,2
3,40,noise 1,0.056231,18.526362,2
0,40,myokit.tumour_volume,0.137718,18.520070,3
1,40,myokit.lambda_0,0.284953,18.520070,3
